# Title

## Setup

In [1]:
import requests
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('.env')
ckey = os.getenv('ckey')
auth = (ckey, '')

In [3]:
chainName = 'eth-mainnet'
ticker1 = input('Give Ticker 1: ') # e.g. UNI (I think ideally put less common one first so you request less data?)
ticker2 = input('Give Ticker 2: ') # e.g. ETH
dexName = input('Give DEX Name: ') # e.g. sushiswap, uniswap_v2
addressByToken = {'UNI': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
                  'WETH': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                  'ETH': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
                 }
try:
    tokenAddress1 = addressByToken[ticker1]
    tokenAddress2 = addressByToken[ticker2]
    print(f'Looking at {ticker1}-{ticker2} pool on {dexName}')
except KeyError:
    print('unknown token')
    tokenAddress1 = input('Give Token Address 1: ')
    tokenAddress2 = input('Give Token Address 2: ')

Give Ticker 1:  UNI
Give Ticker 2:  ETH
Give DEX Name:  sushiswap


Looking at UNI-ETH pool on sushiswap


## Get XY=K transactions for token address

In [4]:
url = f"https://api.covalenthq.com/v1/{chainName}/xy=k/{dexName}/tokens/address/{tokenAddress1}/transactions/"
print(url)

headers = {
    "Content-Type": "application/json",
}
# TODO: Thomas says put params in header to make them work

params = {'page-size': '1000', 'page-number': '0'}

r = requests.get(url, params=params, headers=headers, auth=auth)

print(r.text)


https://api.covalenthq.com/v1/eth-mainnet/xy=k/sushiswap/tokens/address/0x1f9840a85d5af5bf1d1762f925bdaddc4201f984/transactions/
{"data":{"updated_at":"2023-03-05T07:20:32.240006864Z","chain_id":1,"chain_name":"eth-mainnet","items":[{"block_signed_at":"2023-03-03T08:04:23Z","tx_hash":"0x97d50c20e721632d0b6875fc12f04eae7d3f8e83eeaf0303d9ce7cfadfd77612","act":"REMOVE_LIQUIDITY","address":"0xdafd66636e2561b0284edde37e42d192f2844d40","amount_0":"100000000000000000","amount_1":"396941447359274","amount_0_in":null,"amount_1_in":null,"amount_0_out":null,"amount_1_out":null,"to_address":null,"from_address":"0xec08e38a93288b248f988b5e1152c12f65e7b5c8","sender_address":"0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f","total_quote":1.3058075,"quote_currency":"USD","token_0":{"contract_decimals":18,"contract_name":"Uniswap","contract_ticker_symbol":"UNI","contract_address":"0x1f9840a85d5af5bf1d1762f925bdaddc4201f984","supports_erc":null,"logo_url":"https://logos.covalenthq.com/tokens/0x1f9840a85d5af5b

In [5]:
# TODO: ^ covalent thinks has_more=false, idk why

In [6]:
response = json.loads(r.text)

In [7]:
def getAddress(txid: str) -> str:
    url = f"https://api.covalenthq.com/v1/{chainName}/transaction_v2/{txid}/"
    response = requests.get(url, headers=headers, auth=auth)
    response = json.loads(response.text)
    return response['data']['items'][0]['from_address']

In [8]:
for x in response['data']['items']:
    if x['token_1']['contract_address'] == tokenAddress2 \
    and x['act'] == 'SWAP':
        txid = x['tx_hash']
        # need 1 more api request to go from txid --> from_address because api is bad (doesn't give from_address)
        userAddress = getAddress(txid)
        print(txid, userAddress)
        # TODO: filter out bots, getPnl(userAddress), find the top k

0x653607eb7a27ba76b4d2db3fcfc19e208ac26eeb0705e3a36f8569406acb75ec 0x48a61d9870823a392be32bfe207e0803a3c3fffc
0x4df85cfa34f6cb6a825ccfda330445a74e3f995f0427b6fd784785443319c5a8 0xc6c7565644ea1893ad29182f7b6961aab7edfed0
0x8b16d076cc682b19a031c6ba147969ac17be957092a3d74fbc3e5b687d3ff8bd 0x0000e0c70000ca6bc5006838e7cc3e7c8fd23d00
0x101ea53b01b054b258efa19a214ea548e2c777cd2636693f189e2fbef7e621b9 0x76f36d497b51e48a288f03b4c1d7461e92247d5e
0x0993659b72fc6db25f508b2d545c9c79ebbc869059cf606fad8b252ec2e01982 0xad67d3a4fe6040ccb8ae703a443202ec6b9e4a0f
0x9a029a0beeb0dc659ad76c5d3651720357479f3a8de014dcedf89d11d310d103 0x8a79af7f0aaa20685bcfab67d7a49ca52427bef8
0x0b9e9de917425710aef5c8befa816c5815816f7762e4b4ec5924cdf93b30aae1 0xdc58f74ea9338c8eb50ca473cc7afd05d2de487e
0xacc4b2475551998d254eed112f33895698610934e13cecf241ed34d8221d4bd1 0xdc58f74ea9338c8eb50ca473cc7afd05d2de487e
0x5cdb5b6f41b5030107e5a735b606fb7423b2d008088597866c3a058852964184 0xfd32315386252b15fa96f074e98cd5ef799d1fcc
0xb5a0ac10